# Import dep

In [ ]:
import os
import sys
import pandas as pd
import openai
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KE")

current_path = Path.cwd()
src_dir = current_path.parent
sys.path.append(str(src_dir))
# import annotation methods
from src.labels_generator import (generate_relations,relation_search,resort_relation, get_completion,
                                  deserialize_relations,
                                  generate_relations_with_explanation,
                                  relations_tupled,
                                 create_sorted_relation)

# Load matcher
from src.matcher.core import SimCSE_Matcher
matcher = SimCSE_Matcher(str(src_dir/ 'artifacts/matcher_model'))

main_relations = ["supplier", "customer"]
replaces = {"sentence": "{sentence}"}
# Replace the keys with values for unified relation direction
relations_map = {"customer": "supplier"}

# Playground for LLM

In [3]:
data = pd.read_excel('../data/tasks/finetune_llm_on_label_1/source_data.xlsx', index_col="index")


In [4]:
definitions_and_fixes = {
'Add a definition to clarify that "receive payments" refers to CS relation.':\
["- Supplier companies may be referred to as company that recieve payments from customers.", (114)],
'Add rule to capture concept':\
["- If report mention customers or clients consider these as customer relationship"]

}

In [5]:

explanation_prompt = '''
Your task is to provide explanation about the relation between companies mentioned in the report given in ``` quote.
possible relation - {supplier, customer, other_relations, nothing}

##Report
```
{sentence}

```

here is some defintions that might help to understand how to identify relation between companies

## Definitions and concept for `other_relations`
- Investment activity
- Capital exchange
- Asset trade
- Debt and credit relationship
- Working capital transaction
- Non-revenue payment transactions



## Definitions for identifying the supplier or customer relation
- Customer companies may be referred to as a source of revenue or clients or customers for supplier companies
- Customer companies may be referred to as companies that accounts for revenues or net sales of supplier companies
- Customer companies may be referred to as companies that accounts for trade balance, accounts receivable or amount payable of supplier companies
- Customer companies may be referred to as companies that assign or transfer manufacturing responsibilities to suppliers
- Customer companies may be referred to as companies that purchase some product or service to supplier
- Customer companies may be referred to as distributors, user, commercializers, retailers, insurance companies, or similar entities.
- Customer companies could act as a distribution channel. Supplier has original content, and they distribute it via customer companies platform or channel
- Any company the use another company technology or service must be customer of this company
- In collaboration agreement or joint development, customer company will contribute to the supplier companies revenue
- Supplier companies depend on customer companies to supply or sale of their product
- Supplier companies may be referred to as companies that gain or take revenues from customers.
- Supplier companies may be referred as companies whose revenue is increased or decreased or have revenue impact due to customer companies
- Supplier companies may be referred to as those who own, operate, or manufacture materials and then sell to customer companies
- Supplier companies may be referred to as vendors or providers or sellers of services, products, or materials to customer companies
- Supplier companies may say there is `no outstanding receivable balance or payment` from the customer company when all payments are completed. 
- Supplier companies may be referred to as companies whose bill thier customers
- Supplier companies may be referred to as entities that provide goods or services to another company, typically in exchange for payment. They play a crucial role in the supply chain by ensuring the availability of necessary resources for the customer company's operations.
- Supplier companies may be referred to as manufacturers who provide compensating payments for the quantities of products manufactured by alternative manufacturers on behalf of the customer company.
- Supplier companies may be referred to as companies that are eligible for potential milestone payments and royalties based on the commercialization of products arising from research under a collaboration agreement.

## Complex definitions for financial_trade, supplier or customer relation
- If two companies are involved in collaboration agreement or joint development, then think in following steps -
  1. find which company is paying money to another company in joint development
  2. if it is not clear who is paying money, then relation is other_relations
  3. if it is clear who is payee, the payee is customer



## Output example
# relation #1
explanation: explain with clarity who is supplier/customer/finantial_trade and to whom
span with definition: {"span": the text span that indicate certain definition, "definition": the definition text on which relation depends on}
relation phrase: Company_X is supplier/customer/finantial_trade of company_Y 


  
## definitions to follow for creating output -
- Use only the information provided in the report.
- Summarize your answer, making sure it is related to the defined definitions only.
- Do not infer anything outside of the given context.
- Check all definitions, and think step by step as mentioned for complex relations.
- Mention the definition that has been used for your explanation

'''


## Rules generated by the LLMInstructor
- Supplier companies may be referred to as the entities that provide products or services to a specific customer company, contributing to a portion of the customer company's total revenues.
- Supplier companies may be referred to as entities that are able to sell their accounts receivable from a specific customer at a discount to a third-party financial institution. 
- Supplier companies may be referred to as companies that are eligible for potential milestone payments and royalties based on the commercialization of products arising from research under a collaboration agreement. => 189

In [6]:
relation_prompt = '''
Your task is to identify the relation between companies mentioned in the report given in ``` quote which explain the relations between companies.
possible relation - {supplier ,customer , financial_trade, nothing}


##Explanation
```
{explanation}

```

## Rules to follow for creating output -
- use only information given in the explanation
- Don't infer anything outside of given explanation
- Output should be strictly in JSON Object with each json having keys company_1, relation and company_2
- Make sure to includet the direct naming of the relation between each pairs in the output


## output
Return JSON Object contains relations with following -
[{'company_1': Company_X, 
'relation: what <company_1> represent to <company_2> (supplier , customer, financial_trade or nothing),
'company_2': Company_Y,
'reason': why Company_X is supplier or customer/finantial_trade of Company_Y based on the explanation ]

'''

prompt_2 = '''Your task is to identify the relation between companies mentioned in the report given in ``` quote which explain the relations between companies.
possible relation - {supplier ,customer , financial_trade, nothing}


##Explanation
```
{explanation}

```

## Rules to follow for creating output -
- use only information given in the explanation
- Don't infer anything outside of given explanation
- Output should be strictly in JSON Object with each json having keys company_1, relation and company_2


## output
Return single JSON Object that contains relations with the following format in ``` quote -
```{'customer_supplier_relations' : [ {'is customer': Company_X acting as customer, 'is supplier': Company_Y acting as supplier of the customer, 'relation phrase': print the relation phrase from the explanation} ],
   'financial_trade': [ [Company_X, Company_Y] , [Company_X, Company_Y] ], 
   'nothing': [ [Company_X, Company_Y] , [Company_X, Company_Y]]}```

'''

In [7]:
improve_write = """
Your task is to improve the writing in the report given in ``` quote

## Rules to follow for creating output 
- Make the sentence clear and straight forward

## Report
{sentence}

## Output

Report: ..

"""


In [8]:
rule_prompt= """
Your task is to create definition from the following report which associated with supply chain relation fact

## Report
Under the Company_X Agreement as amended Company_Y are eligible for potential pre commercialization and commercialization milestone payments of over $650.0 million in the aggregate on omecamtiv mecarbil and other potential products arising from research under the collaboration, and royalties that escalate based on increasing levels of annual net sales of products commercialized under the agreement.

## Fact
- Company_Y is supplier of Company_X

## Rules to follow for creating output 
- Write defintion that can be used to decribe the fact
- Don't include company names and make the definition as general as possible
- Phrase the definition with this intro `Supplier companies may be referred to as ...` or `Customer companies may be referred to as ...`
- Summarize the ourput
"""

In [24]:
get_completion(rule_prompt)

'Supplier companies may be referred to as companies that are eligible for potential milestone payments and royalties based on the commercialization of products arising from research under a collaboration agreement.'

In [71]:
i = 189
sentence = data.loc[i].sentence
sme_relations = data.loc[i].sme_relations
print(sentence)
print(sme_relations)

Under the Amgen Agreement as amended CYTOKINETICS Inc are eligible for potential pre commercialization and commercialization milestone payments of over $650.0 million in the aggregate on omecamtiv mecarbil and other potential products arising from research under the collaboration, and royalties that escalate based on increasing levels of annual net sales of products commercialized under the agreement.
['CYTOKINETICS INC', 'supplier', 'Amgen']


In [28]:
new_sentence  = get_completion(improve_write.replace("{sentence}", sentence))
new_sentence

'Report: According to this agreement, SCOTTS LIQUID GOLD Inc was able to sell accounts receivable from Wal Mart to Wells Fargo at a discounted price.'

In [69]:
sentence = 'Under the Company_S Agreement as amended Company_Z are eligible for potential pre commercialization and commercialization milestone payments of over $650.0 million in the aggregate on omecamtiv mecarbil and other potential products arising from research under the collaboration, and royalties that escalate based on increasing levels of annual net sales of products commercialized under the agreement.'

In [74]:
explanation = get_completion(explanation_prompt.replace("{sentence}",  sentence))
print(explanation)

# relation #1
explanation: CYTOKINETICS Inc is eligible for potential milestone payments and royalties based on the commercialization of products arising from research under the Amgen Agreement.
span with definition: {"span": "Under the Amgen Agreement as amended", "definition": "Supplier companies may be referred to as companies that are eligible for potential milestone payments and royalties based on the commercialization of products arising from research under a collaboration agreement."}
relation phrase: CYTOKINETICS Inc is a supplier of potential milestone payments and royalties to Amgen.


In [31]:
completion = get_completion(prompt=relation_prompt.replace("{explanation}", explanation))
relations_list = deserialize_json_dict2(completion)
relations = relations_tupled(relations_list, relations_map)
relations

08/14/2023 11:44:37 - WARNING - urllib3.connectionpool -   Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd15af16730>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v1/chat/completions
08/14/2023 11:44:37 - WARNING - urllib3.connectionpool -   Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd15af16970>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /v1/chat/completions
08/14/2023 11:44:37 - WARNING - urllib3.connectionpool -   Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd15afd60d0>: Failed to establish a new conne

[['Company One', 'supplier', 'Company Two']]

In [68]:
from pprint import pprint
pprint(completion)

("[{'company_1': 'Differential Brands Group Inc', 'relation': 'supplier', "
 "'company_2': 'Macy Inc', 'relation text': 'Differential Brands Group Inc is "
 "a supplier to Macy Inc'}, {'company_1': 'Differential Brands Group Inc', "
 "'relation': 'supplier', 'company_2': '...")


In [138]:
print(row['explanation'])
print(row['sme_relations'])


# relation 1
explanation: SONUS NETWORKS Inc has been selected for network deployments by operators including ATT, Belgacom, BT Group, Cable and Wireless International, Carphone Warehouse, CITIC 1616, France Telecom, Global Crossing, KDDI, Level 3, NTT Communications, Qwest, Softbank Corporation, T Systems Business Services a division of Deutsche Telekom Group, Verizon and XO Communications. Based on the information provided, SONUS NETWORKS Inc is being selected by these operators for network deployments. This indicates that SONUS NETWORKS Inc is a supplier of network deployment services to these operators.

definition with span: {"span": "SONUS NETWORKS Inc has been selected for network deployments by operators including ATT, Belgacom, BT Group, Cable and Wireless International, Carphone Warehouse, CITIC 1616, France Telecom, Global Crossing, KDDI, Level 3, NTT Communications, Qwest, Softbank Corporation, T Systems Business Services a division of Deutsche Telekom Group, Verizon and XO

In [75]:
completion = get_completion(prompt=prompt_2.replace("{explanation}", explanation))

In [76]:
print(completion)

{'customer_supplier_relations': [{'is customer': 'Amgen', 'is supplier': 'CYTOKINETICS Inc', 'relation phrase': 'CYTOKINETICS Inc is a supplier of potential milestone payments and royalties to Amgen.'}], 'financial_trade': [], 'nothing': []}


In [77]:
relations_list = deserialize_json_dict2(completion)
relations = relations_tupled_2(relations_list, relations_map)
print("completion", completion)
print("relations", relations) 

completion {'customer_supplier_relations': [{'is customer': 'Amgen', 'is supplier': 'CYTOKINETICS Inc', 'relation phrase': 'CYTOKINETICS Inc is a supplier of potential milestone payments and royalties to Amgen.'}], 'financial_trade': [], 'nothing': []}
relations [['CYTOKINETICS Inc', 'supplier', 'Amgen']]


In [114]:
completion

"{'customer_supplier_relations': [{'customer': 'AMD', 'supplier': 'Rambus Inc'}, {'customer': 'Broadcom', 'supplier': 'Rambus Inc'}, {'customer': 'Cisco Freescale', 'supplier': 'Rambus Inc'}, {'customer': 'Fujitsu', 'supplier': 'Rambus Inc'}, {'customer':..."

In [116]:
for i,c in enumerate(completion[::-1]):
    if c == '}':
        solve = completion[:-i] + ']}'
        break
        

In [74]:
deserialize_json_dict2(completion)

{'customer_supplier_relations': [{'customer': 'Ascension Health',
   'supplier': 'Accretive Health Inc'}],
 'financial_trade': [],
 'nothing': []}

In [19]:
# Read LLM explanined dataset
glob("../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4*.xlsx")

['../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_0_10.xlsx',
 '../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_100_200.xlsx',
 '../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_10_100.xlsx',
 '../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_200_300.xlsx',
 '../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_300_400.xlsx',
 '../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v4_400_500.xlsx']

# Detect Conflict

## Label == 1

In [17]:
glob("../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v2.3_*.xlsx")

['../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v2.3_0_500.xlsx']

In [6]:
from glob import glob
files =glob("../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v2.3_*.xlsx")
output  = pd.concat([pd.read_excel(file, index_col="index") for file in files], axis=0)
print("output.shape",output.shape)
print("output.columns",output.columns)

output.shape (500, 12)
output.columns Index(['sentence', 'Label', 'org_groups', 'inf_relations', 'entity_1',
       'entity_2', 'sme_relations', 'explanation', 'relation_completion',
       'relations', 'completion', 'align'],
      dtype='object')


In [7]:
def eval_relations(relation):
    try:
        re = eval(relation)
    except:
        re = []
    return re

In [8]:
eval(output['relations'].iloc[0])

[['AAC Holdings Inc', 'supplier', 'Blue Cross Blue Shield of California'],
 ['AAC Holdings Inc', 'supplier', 'Anthem Blue Cross Blue Shield of Colorado'],
 ['AAC Holdings Inc', 'supplier', 'United Behavioral Health']]

In [16]:
def eval_relations(relation, default=[]):
    try:
        re = eval(relation)
    except:
        re = default
    return re

def eval_relation_data(output, relations_map):
    # Resort the sme_relations to unify the relations directions
    if not isinstance(output['sme_relations'].iloc[0], list):
        tqdm.pandas(desc="Eval sme_relations")
        output['sme_relations'] = output['sme_relations'].progress_apply(eval)
    if not isinstance(output['relations'].iloc[0], list):
        tqdm.pandas(desc="Eval relations")
        output['relations'] = output['relations'].progress_apply(eval_relations)
    if not isinstance(output['org_groups'].iloc[0], dict):
        tqdm.pandas(desc="Eval org_groups")
        output['org_groups'] = output['org_groups'].progress_apply(eval_relations, default={})  
    tqdm.pandas(desc="Resort sme relations")
    output['sme_relations'] = output['sme_relations'].progress_apply(lambda x:\
                              resort_relation((x[0], x[1], x[2]),
                                            relations_map))

In [11]:
eval_relation_data(output, relations_map)

Resort sme relations: 100%|██████████| 500/500 [00:00<00:00, 284629.75it/s]


In [12]:
# Search relations and return mask
tqdm.pandas(desc="Search relations")
output['align'] =\
output[['sme_relations', 'relations']]\
.progress_apply(lambda x:
relation_search(
query_relation=x[0],
relations_tuples=x[1],
matcher=matcher,
threshold=0.8,
main_relations=list(relations_map.values()) ),axis=1).to_list()

Search relations: 100%|██████████| 500/500 [00:06<00:00, 74.17it/s]


In [13]:
pd.set_option("display.max_colwidth", None)
print("Errors count, Errors rate")
output[output['align'] == False].shape[0],\
output[output['align'] == False].shape[0] / len(output)

Errors count, Errors rate


(166, 0.332)

In [14]:
output[output['align'] == True].shape[0]

334

In [15]:
errors = output[(output['relations'].apply(len) == 0) & (output['relation_completion'].str.contains("supplier")) ] 

In [30]:
errors = output[output.relations.apply(len) == 0 ]
errors = errors[~errors['explanation'].str.contains("nothing")]

# **Transform LLM Data into RE Data**

### **_Load NER Tagger_**

In [ ]:
from src.language_model.spacy_loader import SpacyLoader
spacy_loader = SpacyLoader("en_core_web_trf",
                           entity_matcher="../artifacts/matcher_model/",
                          load_matcher=True)

### **_Load project configuration_**basic_columns

In [39]:
# Read project configuration
import yaml
from src.utils.data import dict2dot


with open(src_dir/ 'params.yaml' ,'r') as o:
    config = dict2dot(yaml.safe_load(o))



ImportError: cannot import name 'dict2dot' from 'src.utils.data' (/notebooks/test/inferess-relation-extraction/src/utils/data.py)

### **_Helper Functions_**

In [4]:

def eval_relations(relation, default=[]):
    try:
        re = eval(relation)
    except:
        re = default
    return re

def eval_relation_data(output, relations_map):
    # Resort the sme_relations to unify the relations directions
    if not isinstance(output['sme_relations'].iloc[0], list):
        tqdm.pandas(desc="Eval sme_relations")
        output['sme_relations'] = output['sme_relations'].progress_apply(eval)
        
        
    if 'relations' in output.columns:
        if not isinstance(output['relations'].iloc[0], list):
            tqdm.pandas(desc="Eval relations")
            output['relations'] = output['relations'].progress_apply(eval_relations)
    else:
        output['relations'] = None
    if not isinstance(output['org_groups'].iloc[0], dict):
        tqdm.pandas(desc="Eval org_groups")
        output['org_groups'] = output['org_groups'].progress_apply(eval_relations, default={})  
    tqdm.pandas(desc="Resort sme relations")
    output['sme_relations'] = output['sme_relations'].progress_apply(lambda x:\
                              resort_relation((x[0], x[1], x[2]),
                                            relations_map))
def process_labeled_data(data,
                         text_col:str,
                         relation_tuple:str,
                        relations_map:dict): 
    # Create org_groups if not exist

    sents, spans, org_groups, aliases = spacy_loader.predictor(data[text_col])
    data[text_col]= sents
    data.loc[:, text_col] = sents
    data.loc[:, "spans"] = spans
    data.loc[:, "org_groups"] = org_groups
    # deserialize data
    eval_relation_data(data, relations_map)
    



In [45]:

import re
import json
import time
from typing import Tuple, List, Text, Dict
from collections import defaultdict
from tqdm import tqdm
import traceback
from typing import Text 
from itertools import chain
from copy import copy
import pandas as pd
import numpy as np
import random
from src.labels_generator.utils import check_relation_tuples


def get_other_relations(org_groups, max_others:int):
    """
    Returns a list of other relations between companies based on the dictionary of company groups passed as input.
    The maximum number of other relations is determined by the max_others parameter.

    @params
    -------
    org_groups (dict): A dictionary with company names as keys and groups as values.
    max_others (int): The maximum number of other relations to return.

    @returns
    --------
    list: A list of other relations between companies.
    """
    # Collect all unique company names from the org_groups dictionary
    orgs = list(org_groups.keys())
    orgs = {k:None for k in set(orgs)}

    # Create a dictionary to map each unique company name to an ID
    orgs_ids = {k:i for i,k in enumerate(set(orgs))}
    ids2org = {i:k for i,k in enumerate(set(orgs_ids))}

    # Create a list of all possible pair-wise combinations of the values in 'ids2org', and randomly choose max_others of those combinations
    available_relations = []
    comp_keys = list(ids2org.keys())
    for i in range(len(comp_keys)):
        for j in range(i+1, len(comp_keys)):
            relation_t = tuple(sorted([comp_keys[i], comp_keys[j]]))
            available_relations.append(relation_t)

    other_ids = set(available_relations)

    other_relations = []
    if len(other_ids) > max_others:
        other_ids = random.sample(other_ids, max_others)

    # Map the IDs back to the company names and create a list of other relations
    for pair in other_ids:
        c1 = ids2org[pair[0]]
        c2 = ids2org[pair[1]]
        other_relations.append((c1,'other', c2))

    return other_relations

def extract_relations_from_llm(datapoint,
                                matcher,
                                text_col:str="sentence",
                                relations_key:str='relations',
                                threshold:float=0.9,
                               max_others=3):
    
    """
    Create a dataset for relation extraction training.

    @params
    -------
    datapoint: A dictionary containing the following keys:
        - org_groups: A dictionary of company names associated with an integer identifier.
        - relations: A list of tuples representing the relationship between companies.
    matcher: An instance of the `StringMatcher` class.
    text_col: The key in the datapoint dictionary where the text to match can be found.
    relations_key: The key in the datapoint dictionary where the relations can be found.
    threshold: The similarity threshold for matching company names.

    @returns
    --------
    llms_relations: A list of tuples representing the relationships between companies that were successfully matched.
    other_relations: A list of tuples representing the relationships between companies that were not matched.

    @raises
    -------
    - ValueError: If the relations list in the datapoint is invalid.
    """
    org_groups = datapoint['org_groups']
    relations = datapoint[relations_key]
    # Assert the relations on the right format
    if not check_relation_tuples:
        raise ValueError("Invlid relations list on the datapoint, must be List[Tuple[Text, Text, Text]]")
    matcher_built = False
    # Collect all companies mentioned in the relations and create a dictionary with each unique company as a key
    llms_companies = []
    if isinstance(relations, list):
        for relation in relations:
            llms_companies += [relation[0] , relation[2]]
    llms_companies = {k:None for k in set(llms_companies)}
    llms_ids = {k:i for i,k in enumerate(set(llms_companies))}
    ids_llms = {i:k for i,k in enumerate(set(llms_companies))}
    # Check if each company in the dictionary is mentioned in the sentence, and if not, try to match it with a known organization
    for company in list(llms_companies.keys()):
        if company in datapoint[text_col]:
            llms_companies[company] = company
        elif len(org_groups):
            if matcher_built is False:
                matcher.build_index(list(org_groups.keys()))
                matcher_built = True
            matches = matcher.search(company, threshold=threshold    , top_k = 3)
            if len(matches) > 0:
                llms_companies[company] = matches[0][0]
            else:
                llms_companies.pop(company)
                ids_llms.pop(llms_ids.pop(company))
                
    # Create a dictionary called 'ids2org' that maps each value in 'org_groups' to a list of keys that have that value
    ids2org = defaultdict(lambda : [])
    for key ,val in llms_ids.items():
        ids2org[val].append(key)

    '''
    Create a list of all possible pair-wise combinations of the values in 'ids2org',
    and randomly choose 5 of those combinations
    '''
    availabel_relations = []
    comp_keys = list(ids2org.keys())
    for i in range(len(comp_keys)):
        for j in range(i+1, len(comp_keys)):
            relation_t = tuple(sorted([comp_keys[i], comp_keys[j]]))
            availabel_relations.append(relation_t)
    exist_relations = []
    llms_relations = []
    if isinstance(relations, list):
        for relation in relations:
            c1, c1_name = relation[0], llms_companies.get(relation[0])
            c2, c2_name = relation[2], llms_companies.get(relation[2])
            relation = relation[1]
            if not all([c1 in llms_companies.keys() , c2 in llms_companies.keys()]):
                continue
            if not all([c1,c2,relation]):
                continue 
            llms_relations.append((c1_name, relation, c2_name))

            c1_id = llms_ids.get(c1)
            c2_id = llms_ids.get(c2)    
            exist_relations.append(tuple(sorted([c1_id, c2_id])))
    
    other_ids = set(availabel_relations) ^ set(exist_relations)
    other_relations = []
    for pair in list(other_ids): 
        c1 = llms_companies[ids_llms[pair[0]]]
        c2 = llms_companies[ids_llms[pair[1]]]    
        other_relations.append((c1,'other', c2))
        
    if len(llms_relations) == 0 and len(other_relations) == 0:
        other_relations = get_other_relations(org_groups, max_others)
        
    return llms_relations, other_relations
def create_re_dataset(
                      output,
                      matcher,
                      text_col:str="sentence",
                      relations_key:str='relations',
                      threshold:float=0.9,
                      max_others:int=3,
                      basic_columns:list=[],
                      ) -> pd.DataFrame:
    """
    Create a relation extraction dataset.

    @params
    -------
    - output: A pandas dataframe containing the following keys:
        * sentence: A string representing a sentence containing relevant text.
        * relations: A list of tuples representing the relationship between companies.
    - matcher: An instance of the `StringMatcher` class.
    - text_col: The key in the output dataframe where the text to match can be found.
    - relations_key: The key in the output dataframe where the relations can be found.
    - threshold: The similarity threshold for matching company names.

    @returns
    --------
    - dataset: A pandas dataframe containing the following columns:
    - idx: A unique identifier for the datapoint.
    - sentence: A string representing the sentence containing relevant text.
    - entity_2: A string representing the second entity in the relation.
    - relation: A string representing the relation between the two entities.
    - entity_1: A string representing the first entity in the relation.
    """

    # Apply the `extract_relations_from_llm` function to each datapoint in the output dataframe
    results = output.apply(lambda x: extract_relations_from_llm(datapoint=x,
                            matcher=matcher,
                            text_col=text_col,
                            relations_key=relations_key,
                            threshold=threshold), axis=1)

    # Create new columns in the output dataframe to store the results of the `extract_relations_from_llm` function
    output['llm_relations']  = results.apply(lambda x : x[0])
    output['other_relations']  = results.apply(lambda x : x[1])
    relation_columns = ['llm_relations', 'other_relations']
    # Create a list of all possible pair-wise combinations of the values in 'ids2org', and randomly choose 5 of those combinations
    columns = relation_columns + [text_col] + basic_columns
    re_dataset = []
    for i, row in output[columns].iterrows():
        row = row.to_dict()
        for r_column in relation_columns:
            # Iterate over relations and ingest row for each relation
            for relation_tuple in row[r_column]:
                row['entity_2'] = relation_tuple[0]
                row['relation'] = relation_tuple[1]
                row['entity_1'] = relation_tuple[2]
                re_dataset.append(dict(row))

    # Return a pandas dataframe containing the relation extraction dataset
    dataset = pd.DataFrame(re_dataset)[[ text_col,
                                        'entity_2',
                                        'relation',
                                        'entity_1'] + basic_columns]
    return dataset


### **_Define and read data files_**

In [6]:
# Define all the files names
huge_1 = 'huge_train_llm_aligned_v2_3_0_1300'
huge_0_complex = "huge_other_train_complex_sents"
huge_1_complex = "huge_train_complex_sents_llm_v2_3"
all_1 = "llm_relations_all_label_1_v2_3"
all_0 = "llm_relations_other_from_label_0_v2_3"
all_other = "llm_relations_other_relation_v2_3"


In [7]:
# declare dict to store dirs and data
data_files = {
    'huge_1': {"dir":src_dir/ "data/raw/{}.xlsx".format(huge_1)},
    'huge_0_complex': {"dir":src_dir/ "data/raw/{}.xlsx".format(huge_0_complex)},
    'huge_1_complex': {"dir":src_dir/ "data/raw/{}.xlsx".format(huge_1_complex)},
    'all_1': {"dir":src_dir/ "data/raw/{}.xlsx".format(all_1)},
    'all_0': {"dir":src_dir/ "data/raw/{}.xlsx".format(all_0)},
    'all_other': {"dir":src_dir/ "data/raw/{}.xlsx".format(all_other)}
}

In [8]:
# method to add Label based on the relations columns
def sc_label_from_relations(relation_tuples, main_relations):
    if not relation_tuples:
        return 0
    if len(relation_tuples) == 0:
        return 0
    for relation_tuple in relation_tuples:
        if len(relation_tuple) != 3:
            continue
        elif  relation_tuple[1] in main_relations:
            return 1
    return 0


In [9]:
sc_label_from_relations(None, main_relations)

0

### **_Prepare data to be in `RE_FORM`_**

In [19]:
# read file and store each into it's coresponding key_value
for key in list(data_files.keys()):
    print(key,'\n--------------')
    data_files[key]['data'] = pd.read_excel(data_files[key]['dir'])
    process_labeled_data(data_files[key]['data'], "sentence", "relations", relations_map)
    data_files[key]['data']['idx'] = ["{}_{}".format(key, i)\
                                      for i in range(data_files[key]['data'].shape[0])]
    data_files[key]['data']['Label'] = data_files[key]['data']['relations']\
                                        .apply(lambda x : sc_label_from_relations(x, main_relations))

huge_1 
--------------
| ID | GPU | MEM |
------------------
|  0 |  0% |  8% |
2023-09-13 11:23:15,065 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-13 11:23:15,066 — 🌌 spaCy — INFO — process chunk#1 ...


Resort sme relations: 100%|██████████| 733/733 [00:00<00:00, 314776.78it/s]


huge_0_complex 
--------------
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
2023-09-13 11:23:28,470 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-13 11:23:28,471 — 🌌 spaCy — INFO — process chunk#1 ...


Resort sme relations: 100%|██████████| 2609/2609 [00:00<00:00, 582691.11it/s]


huge_1_complex 
--------------
| ID | GPU | MEM |
------------------
|  0 |  0% | 15% |
2023-09-13 11:23:44,614 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-13 11:23:44,615 — 🌌 spaCy — INFO — process chunk#1 ...


...

Resort sme relations: 100%|██████████| 1962/1962 [00:00<00:00, 580749.78it/s]


all_0 
--------------
| ID | GPU | MEM |
------------------
|  0 |  0% | 15% |
2023-09-13 11:23:59,279 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-13 11:23:59,281 — 🌌 spaCy — INFO — process chunk#1 ...


Resort sme relations: 100%|██████████| 877/877 [00:00<00:00, 480334.89it/s]

all_other 
--------------
| ID | GPU | MEM |
------------------
|  0 |  2% | 15% |
2023-09-13 11:24:02,041 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-13 11:24:02,043 — 🌌 spaCy — INFO — process chunk#1 ...



Resort sme relations: 100%|██████████| 111/111 [00:00<00:00, 229456.75it/s]


In [25]:
for key in list(data_files.keys()):
    print("Columns in {}: {}".format(key,data_files[key]['data'].columns))
    print("-"*60)

Columns in huge_1: Index(['index', 'accessionNumber', 'filer', 'firstEntity', 'relationship',
       'secondEntity', 'sentence', 'sme_relations', 'explanation',
       'relation_completion', 'relations', 'align', 'concepts',
       'concept explanation', 'concept_class', 'spans', 'org_groups', 'idx',
       'Label'],
      dtype='object')
------------------------------------------------------------
Columns in huge_0_complex: Index(['Acc no', 'filer', 'secondEntity', 'sentence', 'Prediction', 'Score',
       'relationship', 'index', 'sme_relations', 'sent_size', 'clause_size',
       'concepts', 'concept explanation', 'concept_class', 'spans',
       'org_groups', 'relations', 'idx', 'Label'],
      dtype='object')
------------------------------------------------------------
Columns in huge_1_complex: Index(['index', 'accessionNumber', 'filer', 'firstEntity', 'relationship',
       'secondEntity', 'sentence', 'sme_relations', 'sent_size', 'clause_size',
       'concepts', 'concept expla

### **_Create `RE_FORM` dataset for each file_**

In [46]:
# read file and store each into it's coresponding key_value
for key in list(data_files.keys()):
    print(key,'\n--------------')
    dataset = create_re_dataset(output=data_files[key]['data'],
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=1,
                           basic_columns=config.base.basic_columns)
    data_files[key]['dataset'] = dataset

09/13/2023 13:18:40 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:40 - INFO - matcher.core -   Building index...
09/13/2023 13:18:40 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:40 - INFO - matcher.core -   Finished
09/13/2023 13:18:40 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:40 - INFO - matcher.core -   Building index...
09/13/2023 13:18:40 - INFO - matcher.core -   Use GPU-version faiss


huge_1 
--------------


09/13/2023 13:18:40 - INFO - matcher.core -   Finished
09/13/2023 13:18:40 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:40 - INFO - matcher.core -   Building index...
09/13/2023 13:18:40 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:40 - INFO - matcher.core -   Finished
09/13/2023 13:18:40 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:40 - INFO - matcher.core -   Building index...
09/13/2023 13:18:40 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:40 - INFO - matcher.core -   Finished
09/13/2023 13:18:40 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:40 - INFO - matcher.core -   Building index...
09/13/2023 13:18:40 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:40 - INFO - matcher.core -   Finished
09/13/2023 13:18:41 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:41 - INFO - matcher.core -   Buildi

huge_0_complex 
--------------
huge_1_complex 
--------------


09/13/2023 13:18:58 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:59 - INFO - matcher.core -   Finished
09/13/2023 13:18:59 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:59 - INFO - matcher.core -   Building index...
09/13/2023 13:18:59 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:59 - INFO - matcher.core -   Finished
09/13/2023 13:18:59 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:59 - INFO - matcher.core -   Building index...
09/13/2023 13:18:59 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:59 - INFO - matcher.core -   Finished
09/13/2023 13:18:59 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:18:59 - INFO - matcher.core -   Building index...
09/13/2023 13:18:59 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:18:59 - INFO - matcher.core -   Finished
09/13/2023 13:18:59 - INFO - matcher.core -   Encoding embeddings f

...

09/13/2023 13:19:27 - INFO - matcher.core -   Finished
09/13/2023 13:19:27 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:27 - INFO - matcher.core -   Building index...
09/13/2023 13:19:27 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:27 - INFO - matcher.core -   Finished
09/13/2023 13:19:27 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:27 - INFO - matcher.core -   Building index...
09/13/2023 13:19:27 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:27 - INFO - matcher.core -   Finished
09/13/2023 13:19:27 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:27 - INFO - matcher.core -   Building index...
09/13/2023 13:19:27 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:27 - INFO - matcher.core -   Finished
09/13/2023 13:19:27 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:27 - INFO - matcher.core -   Buildi

all_0 
--------------


09/13/2023 13:19:49 - INFO - matcher.core -   Finished
09/13/2023 13:19:49 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:49 - INFO - matcher.core -   Building index...
09/13/2023 13:19:49 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:49 - INFO - matcher.core -   Finished
09/13/2023 13:19:49 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:49 - INFO - matcher.core -   Building index...
09/13/2023 13:19:49 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:49 - INFO - matcher.core -   Finished
09/13/2023 13:19:49 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:49 - INFO - matcher.core -   Building index...
09/13/2023 13:19:50 - INFO - matcher.core -   Use GPU-version faiss
09/13/2023 13:19:50 - INFO - matcher.core -   Finished
09/13/2023 13:19:50 - INFO - matcher.core -   Encoding embeddings for sentences...
09/13/2023 13:19:50 - INFO - matcher.core -   Buildi

all_other 
--------------


/tmp/ipykernel_723/815157286.py:51: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  other_ids = random.sample(other_ids, max_others)


In [20]:
print(data_files.keys())

dict_keys(['huge_1', 'huge_0_complex', 'huge_1_complex', 'all_1', 'all_0', 'all_other'])


In [ ]:
s

In [54]:
data_files['all_1']['data'].loc[1878][['sentence', 'llm_relations', 'other_relations']]

sentence                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              In addition, Daimler AG, Fiat Chrysler Automobiles, Hyundai Motor Company, Jaguar Land Rover Autom

In [ ]:
data_files['all_1']['dataset']

In [62]:
all_dataset= pd.concat([data_files[k]['dataset'] for k in ['huge_1', 'huge_0_complex', 'huge_1_complex', 'all_1', 'all_0', 'all_other']], axis=0)

In [63]:
all_dataset['idx'].unique().shape

(7918,)

### **_Check dataset health_**

In [64]:
from src.utils.preprocess import create_re_data, word_search, Intersection


In [65]:
founded1 = all_dataset.apply(lambda x : word_search(x['entity_1'], x['sentence']), axis =1 )
founded2 = all_dataset.apply(lambda x : word_search(x['entity_2'], x['sentence']), axis =1 )

In [66]:
(founded1.apply(len) == 0).sum(),(founded2.apply(len) == 0).sum()


(0, 0)

In [101]:
all_dataset

,idx,sentence,entity_2,relation,entity_1,Label,org_groups,spans
0,huge_1_0,"In addition, five customers, including Sony an...",ADVANCED MICRO DEVICES Inc,supplier,Sony,1,"{'Microsoft': 0, 'Sony': 1, 'ADVANCED MICRO DE...","[{'text': 'five', 'label': 'CARDINAL', 'start'..."
1,huge_1_0,"In addition, five customers, including Sony an...",ADVANCED MICRO DEVICES Inc,supplier,Microsoft,1,"{'Microsoft': 0, 'Sony': 1, 'ADVANCED MICRO DE...","[{'text': 'five', 'label': 'CARDINAL', 'start'..."
2,huge_1_0,"In addition, five customers, including Sony an...",Microsoft,other,Sony,1,"{'Microsoft': 0, 'Sony': 1, 'ADVANCED MICRO DE...","[{'text': 'five', 'label': 'CARDINAL', 'start'..."
3,huge_1_1,"In 2015, Microsoft Corp and Sony Corp each acc...",ADVANCED MICRO DEVICES Inc,supplier,Microsoft Corp,1,"{'Microsoft Corp': 0, 'Sony Corp': 1, 'ADVANCE...","[{'text': '2015', 'label': 'DATE', 'start': 3,..."
4,huge_1_1,"In 2015, Microsoft Corp and Sony Corp each acc...",ADVANCED MICRO DEVICES Inc,supplier,Sony Corp,1,"{'Microsoft Corp': 0, 'Sony Corp': 1, 'ADVANCE...","[{'text': '2015', 'label': 'DATE', 'start': 3,..."
...,...,...,...,...,...,...,...,...
2241,all_0_874,"Following the enactment of the New Laws, CONSO...",WAC,other,CONSOLIDATED WATER Co Ltd,0,"{'Cayman Water’s': 0, 'WAC': 1, 'CONSOLIDATED ...","[{'text': 'the New Laws', 'label': 'LAW', 'sta..."
2242,all_0_875,KLX Inc reports that Shearman Sterling LLP’s o...,KLX Inc,financial_trade,Shearman Sterling LLP,0,"{'KLX Inc': 0, 'BE Aerospace': 1, 'Shearman St...","[{'text': 'KLX Inc', 'label': 'ORG', 'start': ..."
2243,all_0_876,PRICELINE reports that online travel search an...,TripAdvisor,other,PRICELINE,0,"{'TripAdvisor': 0, 'Qunar': 1, 'PRICELINE': 2,...","[{'text': 'PRICELINE', 'label': 'ORG', 'start'..."
2244,all_0_876,PRICELINE reports that online travel search an...,Expedia,other,HotelsCombined,0,"{'TripAdvisor': 0, 'Qunar': 1, 'PRICELINE': 2,...","[{'text': 'PRICELINE', 'label': 'ORG', 'start'..."


In [102]:
founded1.isna().sum()

0

In [103]:
train = create_re_data(
    dataframe = all_dataset,
    text="sentence",
    ent1= "entity_1",
    ent2= "entity_2",
    label= "relation",
    main_relations = ['supplier', 'customer'],
    random_choice = False)

  0%|          | 0/22051 [00:00<?, ?it/s]

In [45]:
sents, spans, org_groups, aliases = spacy_loader.predictor(all_dataset['sentence'].tolist())

| ID | GPU | MEM |
------------------
|  0 |  0% | 24% |
2023-09-11 12:20:03,544 — 🌌 spaCy — INFO — Start batch job for 2 chunks
2023-09-11 12:20:03,545 — 🌌 spaCy — INFO — process chunk#1 ...
2023-09-11 12:21:17,367 — 🌌 spaCy — INFO — process chunk#2 ...


In [13]:
all_dataset['sentence'] = sents
all_dataset['spans'] = spans
all_dataset['org_groups'] = org_groups
all_dataset['aliases'] = aliases

In [15]:
all_dataset['relation'] =  all_dataset['relation'].apply(lambda x : x if x in main_relations else "other")

In [51]:
all_dataset['relation'].value_counts()

other              13263
supplier            7336
financial_trade     1307
nothing              145
Name: relation, dtype: int64

In [67]:
all_dataset.reset_index(drop=True).to_json(src_dir / 'data/raw/llm_aligned_0_1_huge_complex.json')

In [68]:
all_dataset.shape

(22267, 9)

In [13]:
key = "huge_1_complex"
dataset = create_re_dataset(output=data_files[key]['data'],
                       matcher=spacy_loader.entity_matcher,
                       text_col="sentence",
                       relations_key="relations",
                       threshold=0.9,
                       max_others=2)
data_files[key]['dataset'] = dataset

09/08/2023 18:01:53 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:53 - INFO - matcher.core -   Building index...
09/08/2023 18:01:53 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:53 - INFO - matcher.core -   Finished
09/08/2023 18:01:53 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:53 - INFO - matcher.core -   Building index...
09/08/2023 18:01:53 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:53 - INFO - matcher.core -   Finished
09/08/2023 18:01:53 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:53 - INFO - matcher.core -   Building index...
09/08/2023 18:01:53 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:53 - INFO - matcher.core -   Finished
09/08/2023 18:01:53 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:53 - INFO - matcher.core -   Building index...
09/08/2023 18:01:53 - INFO - matcher.core -

error is here huge_1_complex_103 org_groups {}


09/08/2023 18:01:54 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:54 - INFO - matcher.core -   Finished
09/08/2023 18:01:54 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:54 - INFO - matcher.core -   Building index...
09/08/2023 18:01:54 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:54 - INFO - matcher.core -   Finished
09/08/2023 18:01:54 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:54 - INFO - matcher.core -   Building index...
09/08/2023 18:01:54 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:54 - INFO - matcher.core -   Finished
09/08/2023 18:01:54 - INFO - matcher.core -   Encoding embeddings for sentences...
09/08/2023 18:01:54 - INFO - matcher.core -   Building index...
09/08/2023 18:01:54 - INFO - matcher.core -   Use CPU-version faiss
09/08/2023 18:01:54 - INFO - matcher.core -   Finished
09/08/2023 18:01:54 - INFO - matcher.core -   Encoding embeddings f

In [ ]:
# read file and store each into it's coresponding key_value
for key in list(data_files.keys()):
    dataset = create_re_dataset(output=data_files[key]['data'],
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=2)
    data_files[key]['dataset'] = dataset

In [14]:
data_files['huge_1_complex']['data']['Label'].value_counts()

1    1834
Name: Label, dtype: int64

In [ ]:
process_labeled_data(huge_aligned, "sentence", "relations", relations_map)
process_labeled_data(output0, "sentence", "relations", relations_map)
process_labeled_data(output1, "sentence", "relations", relations_map)


In [2]:
output0 = pd.read_excel("../data/raw/llm_relations_other_from_label_0_v2_3.xlsx", index_col='index')

In [3]:
output0['Label'] = 0

In [4]:
output0.shape

(877, 12)

In [9]:
output1 = pd.read_excel('../data/raw/llm_relations_v2_3.xlsx', index_col="index").query("align==True")

In [10]:
output1['Label'] = 1

In [11]:
huge_aligned = pd.read_excel("../data/raw/huge_train_llm_aligned_v2_3_0_1300.xlsx", index_col="index")


In [24]:
huge_aligned['Label'] = 1

2023-09-08 16:12:16,847 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-09-08 16:12:16,848 — 🌌 spaCy — INFO — spaCy Work On GPU


In [18]:
process_labeled_data(huge_aligned, "sentence", "relations", relations_map)
process_labeled_data(output0, "sentence", "relations", relations_map)
process_labeled_data(output1, "sentence", "relations", relations_map)


| ID | GPU | MEM |
------------------
|  0 |  0% | 11% |
2023-09-05 09:23:28,602 — 🌌 spaCy — INFO — Start batch job for 1 chunks
2023-09-05 09:23:28,604 — 🌌 spaCy — INFO — process chunk#1 ...


Resort sme relations: 100%|██████████| 1962/1962 [00:00<00:00, 312280.83it/s]


In [ ]:
dataset0 = create_re_dataset(output=output0,
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=4)
dataset1 = create_re_dataset(output=output1,
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=4)
huge_dataset = create_re_dataset(output=huge_aligned,
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=4)

NameError: name 'output0' is not defined

In [32]:
pd.concat([dataset0, dataset1, huge_dataset], axis=0).reset_index(drop=True)\
.to_json('../data/raw/llm_v2_3_label_0_1_huge_align.json', index="idx")

In [33]:
pd.concat([dataset0, dataset1], axis=0).reset_index(drop=True)\
.to_json('../data/raw/llm_v2_3_label_0_1_align.json', index="idx")

### Merge two sheets

In [ ]:
v_21 = pd.read_excel("../data/tasks/finetune_llm_on_label_1/llm_conflicts_with_sme_v2.xlsx")


In [ ]:
output.to_excel("../data/tasks/finetune_llm_on_label_1/llm_relations_with_explained_v2.2.xlsx")

In [ ]:
v_21['sentence'].isin()

In [ ]:
v_21['sentence'].shape

In [ ]:
(v_21['Label'] == 1).sum()

In [ ]:
source['sentence'].isin(v_21['sentence'].tolist()).sum()

In [ ]:
v_21['sentence']

In [ ]:
v_21[['True label (sme / llm / both / none)', 'Type of llm mistake',
       'Rule to fix the prompt', 'Additional comments']]

In [ ]:
output.loc[v_21.index, ['True label (sme / llm / both / none)', 'Type of llm mistake',
       'Rule to fix the prompt', 'Additional comments']] = v_21[['True label (sme / llm / both / none)', 'Type of llm mistake',
       'Rule to fix the prompt', 'Additional comments']]

In [ ]:
output.loc[v_21.index]

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
v_21[(v_21['relations_v2.1'].str.contains("supplier") ) & (v_21['align_v2.1']==False)].to_excel("../data/tasks/finetune_llm_on_label_1/llm_relations_0_500_v2_1_conflict_direction.xlsx", index_label="index")

In [ ]:
v_21[(v_21['relations_v2.1'].str.contains("supplier")) & v_21['align_v2.1']==False].shape

In [ ]:
v_21[v_21['relations_v2.1'].str.contains("supplier") | v_21['align_v2.1']==False].sample(10)

In [ ]:

output[['explanation_v2.1', 'prompt2_result_v2.1', 'relations_v2.1', 'align_v2.1']] = v_21[['explanation_v2.1', 'prompt2_result_v2.1', 'relations_v2.1', 'align_v2.1']]

### Merge reviewed data from before task

In [ ]:
reviewed = pd.read_excel('../data/tasks/finetune_llm_on_label_1/conflict_llm_with_sme_explained_0_500.xlsx', index_col="index")

In [ ]:
output[['True label (sme / llm / both / none)', 'Type of llm mistake', 'Rule to fix the prompt', 'Additional comments']] = None

In [ ]:
output.loc[reviewed.index , ['True label (sme / llm / both / none)', 'Type of llm mistake', 'Rule to fix the prompt', 'Additional comments']] = reviewed[['True label (sme / llm / both / none)', 'Type of llm mistake', 'Rule to fix the prompt', 'Additional comments']]

In [ ]:
output.to_excel("llm_relations_0_500_v2_and_v2_1.xlsx", index_label="index")


# Analyzing the Non-Conflicted Example

_In this experiment, we aim to validate whether we can trust LLM when it aligns with sme_relation to verify if the rest of the extracted relations are valid for training._

In [ ]:
import pandas as pd
# Read the data
data = pd.read_json("../data/raw/Inferess_train_data.json")
# Read data with Label == 1 or with Label is None 
data = data[(data['Label'] == 1) | data['Label'].isna()]

In [ ]:
data.to_excel("../data/tasks/finetune_llm_on_label_1/source_data.xlsx", index_label="index")

In [ ]:
data  = pd.read_excel("../data/tasks/finetune_llm_on_label_1/source_data.xlsx", index_col="index")

In [ ]:
tqdm.pandas(desc="Create sme_relations column")
data['sme_relations']=data[['entity_2', 'inf_relations', 'entity_1','Label']]\
    .progress_apply(lambda x:\
    create_sorted_relation(x[0],
        x[1],
        x[2],
        x[3],
        relations_map=relations_map),
        axis = 1)

In [ ]:
data['sme_relations'].apply(lambda x : x[1] == 'other').sum()


In [ ]:
data[0:500].shape

In [ ]:
batch = [0,500]
# Generate list of relations in tuples
explantion_replaces = {"sentence": "{sentence}"}
relation_replaces = {"explanation": "{explanation}"}
# Replace the keys with values for unified relation direction
relations_map = {"customer": "supplier"}
# Generate list of relations in tuples
output = generate_relations_with_explanation(data=data[batch[0]:batch[1]],
                       explanation_prompt=explanation_prompt,
                       relation_prompt=relation_prompt,
                       explantion_replaces=explantion_replaces,
                       relation_replaces=relation_replaces,
                       relations_map=relations_map)

In [ ]:
pd.set_option("display.max_colwidth", None)
output.shape

In [ ]:
output.to_excel("../data/tasks/finetune_llm_on_label_1/llm_relations_0_500_v2.xlsx", index_label="index")

In [ ]:
# Resort the sme_relations to unify the relations directions
if not isinstance(output['sme_relations'].iloc[0], list):
    tqdm.pandas(desc="Eval sme_relations")
    output['sme_relations'] = output['sme_relations'].progress_apply(eval)
if not isinstance(output['relations'].iloc[0], list):
    tqdm.pandas(desc="Eval relations")
    output['relations'] = output['relations'].progress_apply(eval)   
tqdm.pandas(desc="Resort sme relations")
output['sme_relations'] = output['sme_relations'].progress_apply(lambda x:\
                          resort_relation((x[0], x[1], x[2]),
                                        relations_map))

In [ ]:
output[['sme_relations', 'relations']]

In [ ]:
# Search relations and return mask
tqdm.pandas(desc="Search relations")
output['align'] =\
output[['sme_relations', 'relations']]\
.progress_apply(lambda x:
relation_search(
query_relation=x[0],
relations_tuples=x[1],
matcher=matcher,
threshold=0.85,
main_relations=list(relations_map.values()) ),axis=1).to_list()


In [ ]:
output['align'].sum() / output.shape[0]

In [ ]:
pd.set_option("display.max_colwidth", None)
output.query("align == False").to_excel("../data/tasks/finetune_llm_on_label_1/conflict_llm_with_sme_0_500.xlsx", index_label="index")